Yiting Wang 4646909520

(a)

In [1]:
import numpy as np, pandas as pd 
data = pd.read_csv('../Data/Anuran Calls (MFCCs)/Frogs_MFCCs.csv')

In [2]:
X = data.iloc[:, :-4]
y = data.iloc[:, -4:-1]

In [4]:
import operator
from sklearn.cluster import KMeans
from sklearn.metrics import hamming_loss, silhouette_score, accuracy_score

monteCarlo_optimalK = {}
for t in range(1,51):
    score_dict = {}
    for k in range(2,51):
        k_means = KMeans(n_clusters = k).fit(X)
        cluster_index = k_means.labels_
        silhouettes = silhouette_score(X, cluster_index)
        score_dict[k] = silhouettes
    
    sort_silhouettes = sorted(score_dict.items(), key = operator.itemgetter(1), reverse = True)
    optimal_k = sort_silhouettes[0][0]
    monteCarlo_optimalK[t] = optimal_k 

In [48]:
k = np.linspace(1, 50)
T = []
for t in k:
    T.append(int(t))
T = pd.DataFrame(T, columns = ['Times'])

optimalK_list = []
for t in k:
    optimalK_list.append(monteCarlo_optimalK[t])
optimalK_list = pd.DataFrame(optimalK_list, columns = ['Optimal k'])

table = pd.concat([T, optimalK_list], axis = 1)
print(table)

    Times  Optimal k
0       1          4
1       2          4
2       3          4
3       4          4
4       5          4
5       6          4
6       7          4
7       8          4
8       9          4
9      10          4
10     11          4
11     12          4
12     13          4
13     14          4
14     15          4
15     16          4
16     17          4
17     18          4
18     19          4
19     20          4
20     21          4
21     22          4
22     23          4
23     24          4
24     25          4
25     26          4
26     27          4
27     28          4
28     29          4
29     30          4
30     31          4
31     32          4
32     33          4
33     34          4
34     35          4
35     36          4
36     37          4
37     38          4
38     39          4
39     40          4
40     41          4
41     42          4
42     43          4
43     44          4
44     45          4
45     46          4
46     47    

(b)

In [9]:
#According to the results, optimal k is 4.
optimal_k = 4

monteCarlo_kIndex = {}
for t in range(1, 51):
    optimal_k = monteCarlo_optimalK[t]
    cluster = KMeans(n_clusters = optimal_k).fit(X)    # build kmeans cluster with optimal k
    cluster_index = cluster.labels_
    monteCarlo_kIndex[t] = cluster_index

In [6]:
family_majority, genus_majority, species_majority = [],[],[]

for t in range(1, 51):
    optimal_k = monteCarlo_optimalK[t]        #optimal k in each time
    cluster_index = monteCarlo_kIndex[t]
    cluster_label = []
    
    #Determine majority in different clusters
    for i in range(0, optimal_k):
        family, genus, species = {},{},{}
        
        for j in range(0, len(y)):
            if cluster_index[j]==i:
                
                #Calculate Family Majority 
                family_class = y['Family'][j]
                if family_class in family.keys():
                    family[family_class] += 1
                else:
                    family[family_class] = 1
                
                #Calculate Genus Majority
                genus_class = y['Genus'][j]
                if genus_class in genus.keys():
                    genus[genus_class] += 1
                else:
                    genus[genus_class] = 1
                
                #Calculate Species Majority
                species_class = y['Species'][j]
                if species_class in species.keys():
                    species[species_class] += 1
                else:
                    species[species_class] = 1
        
        #Sort the label
        sorted_family = sorted(family.items(), key = operator.itemgetter(1), reverse = True)
        sorted_genus = sorted(genus.items(), key = operator.itemgetter(1), reverse = True)
        sorted_species = sorted(species.items(), key = operator.itemgetter(1), reverse = True)
        
        label = {}
        label['Family'] = sorted_family[0][0]
        label['Genus'] = sorted_genus[0][0]
        label['Species'] = sorted_species[0][0]
        cluster_label.append(label)
        
        #Find the majority
        family_majority.append(sorted_family[0][0])
        genus_majority.append(sorted_genus[0][0])
        species_majority.append(sorted_species[0][0])

In [123]:
#Times and Clusters
times, cluster = [], []
k = np.linspace(1,50)
for t in k:
    for i in range(1,5):
        times.append(int(t))
        cluster.append('Cluster ' + str(i))

Times = pd.DataFrame(times, columns = ['Times'])
Cluster  = pd.DataFrame(cluster, columns = ['Cluster'])

#Family Majority, Genus Majority, Species Majority
Family_majority = pd.DataFrame(family_majority, columns = ['Family Majority'])
Genus_majority = pd.DataFrame(genus_majority, columns = ['Genus Majority'])
Species_majority = pd.DataFrame(species_majority, columns = ['Species Majority'])

table = pd.concat([Times, Cluster, Family_majority, Genus_majority, Species_majority], axis = 1)
pd.set_option('display.max_rows', None)
print(table)

     Times    Cluster  Family Majority Genus Majority        Species Majority
0        1  Cluster 1          Hylidae      Hypsiboas    HypsiboasCinerascens
1        1  Cluster 2  Leptodactylidae      Adenomera  AdenomeraHylaedactylus
2        1  Cluster 3    Dendrobatidae       Ameerega      Ameeregatrivittata
3        1  Cluster 4          Hylidae      Hypsiboas       HypsiboasCordobae
4        2  Cluster 1    Dendrobatidae       Ameerega      Ameeregatrivittata
5        2  Cluster 2  Leptodactylidae      Adenomera  AdenomeraHylaedactylus
6        2  Cluster 3          Hylidae      Hypsiboas       HypsiboasCordobae
7        2  Cluster 4          Hylidae      Hypsiboas    HypsiboasCinerascens
8        3  Cluster 1          Hylidae      Hypsiboas       HypsiboasCordobae
9        3  Cluster 2  Leptodactylidae      Adenomera  AdenomeraHylaedactylus
10       3  Cluster 3    Dendrobatidae       Ameerega      Ameeregatrivittata
11       3  Cluster 4          Hylidae      Hypsiboas    Hypsibo

(c)

In [7]:
Hamming_distance_list, Hamming_loss_list, Hamming_score_list = [],[],[]
 
for t in range(1, 51):
    cluster_index = monteCarlo_kIndex[t]
    family_pred, genus_pred, species_pred = [],[],[]
    
    for i in range(len(X)):
        cluster_name = cluster_index[i]
        label = cluster_label[cluster_name]
        family_pred_label = label['Family']
        genus_pred_label = label['Genus']
        species_pred_label = label['Species']

        family_pred.append(family_pred_label)
        genus_pred.append(genus_pred_label)
        species_pred.append(species_pred_label)
    
    #Family, Genus and Species
    family = np.array(cluster_pred['Family'])
    genus = np.array(cluster_pred['Genus'])
    species = np.array(cluster_pred['Species'])
    
    #Prediction of Family, Genus and Species, y
    family_pred = np.array(family_pred)
    genus_pred = np.array(genus_pred)
    species_pred = np.array(species_pred)
    
    #Family Hamming Loss and Family Hamming Score
    family_Hamming_loss = hamming_loss(family_pred, family)
    family_hamming_score = 1 - family_Hamming_loss
    
    #Genus Hamming Loss and Genus Hamming Score
    genus_Hamming_loss = hamming_loss(genus_pred, genus)
    genus_hamming_score = 1 - genus_Hamming_loss
    
    #Species Hamming Loss and Species Hamming Score
    species_Hamming_loss = hamming_loss(species_pred, species)
    species_hamming_score = 1 - species_Hamming_loss
    
    #Hamming Distance, Hamming Loss, Hamming Score
    Hamming_distance = family_Hamming_loss + genus_Hamming_loss + species_Hamming_loss
    Hamming_distance = round(Hamming_distance, 3)
    
    Hamming_loss = Hamming_distance/3
    Hamming_loss = round(Hamming_loss, 3)
    
    Hamming_score = (family_hamming_score + genus_hamming_score + species_hamming_score)/3
    Hamming_score = round(Hamming_score, 3)
    
    Hamming_distance_list.append(Hamming_distance)
    Hamming_loss_list.append(Hamming_loss)
    Hamming_score_list.append(Hamming_score)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [119]:
#Hamming Distance, Hamming Loss, Hamming Score
Hamming_Distance = pd.DataFrame(Hamming_distance_list, columns = ['Hamming Distance'])
Hamming_Loss = pd.DataFrame(Hamming_loss_list, columns = ['Hamming Loss'])
Hamming_Score = pd.DataFrame(Hamming_score_list, columns = ['Hamming Score'])

table = pd.concat([T, Hamming_Distance, Hamming_Loss, Hamming_Score], axis = 1)
print(table)

    Times  Hamming Distance  Hamming Loss  Hamming Score
0       1             0.442         0.147          0.853
1       2             0.387         0.129          0.871
2       3             0.387         0.129          0.871
3       4             0.812         0.271          0.729
4       5             0.353         0.118          0.882
5       6             0.387         0.129          0.871
6       7             0.935         0.312          0.688
7       8             0.935         0.312          0.688
8       9             0.846         0.282          0.718
9      10             0.783         0.261          0.739
10     11             0.353         0.118          0.882
11     12             0.442         0.147          0.853
12     13             0.351         0.117          0.883
13     14             0.387         0.129          0.871
14     15             0.918         0.306          0.694
15     16             0.387         0.129          0.871
16     17             0.353    

In [125]:
table = pd.DataFrame(np.zeros((2,3)))
table.columns = ['Hamming Distance', 'Hamming Loss', 'Hamming Score']
table.index = ['average', 'std']

mean_Hamming_d = np.mean(Hamming_Distance)
mean_Hamming_d = round(mean_Hamming_d, 3)
table.iloc[0,0] = float(mean_Hamming_d)

std_Hamming_d = np.std(Hamming_Distance)
std_Hamming_d = round(std_Hamming_d, 3)
table.iloc[1,0] = float(std_Hamming_d)

mean_Hamming_l = np.mean(Hamming_Loss)
mean_Hamming_l = round(mean_Hamming_l, 3)
table.iloc[0,1]  = float(mean_Hamming_l)

std_Hamming_l = np.std(Hamming_Loss)
std_Hamming_l = round(std_Hamming_l, 3)
table.iloc[1,1] = float(std_Hamming_l)

mean_Hamming_s = np.mean(Hamming_Score)
mean_Hamming_s = round(mean_Hamming_s, 3)
table.iloc[0,2] = float(mean_Hamming_s)

std_Hamming_s = np.std(Hamming_Score)
std_Hamming_s = round(std_Hamming_s, 3)
table.iloc[1,2] = float(std_Hamming_s)

print(table)

         Hamming Distance  Hamming Loss  Hamming Score
average             0.593         0.198          0.802
std                 0.246         0.082          0.082
